In [1]:
%reset -f
# Import functions from the module
from netcdf_tools import get_points, get_values, select_netCDf
import netCDF4 as nc

In [2]:
import pandas as pd

df = pd.read_csv('../data/trajectories.csv')

trajectories_df = pd.DataFrame(df)

trajectories_date = trajectories_df['date'].unique()


In [6]:

end_of_string = '_FNMOC_10m_Surface_Winds.nc'
data_path = '../data/wind/'
date_of_trajectory = trajectories_date[0]
nc_file = select_netCDf(data_path,end_of_string,date_of_trajectory)
nc_file.variables.keys()

../data/wind/20140119_FNMOC_10m_Surface_Winds.nc


dict_keys(['time', 'altitude', 'latitude', 'longitude', 'wnd_ucmp_height_above_ground', 'wnd_vcmp_height_above_ground'])

In [ ]:
import glob as glob


files = glob.glob('../data/wind/2014*.nc')
print(len(files))
wind_data = nc.Dataset(files[0])
time_value = wind_data.variables['time'].actual_range[0]
date_time = nc.num2date(time_value, wind_data.variables['time'].units).strftime('%Y-%m-%d')
date_time

In [ ]:
daily_trajectories = get_points(trajectories_df, date_time)
daily_trajectories

In [ ]:
wind_data.variables.keys()

In [ ]:
import xarray as xr
import numpy as np

data = xr.open_dataset(files[0])
data.keys()


In [ ]:
wind_units = data['wnd_ucmp_height_above_ground'].units
wind_date = data['time'].values
wind_date = pd.to_datetime(wind_date.astype('datetime64[s]')).strftime('%Y-%m-%d').values[0]

print(wind_units)
print(wind_date)

In [ ]:
lon = data['longitude'].values
lat = data['latitude'].values
u_wind = data['wnd_ucmp_height_above_ground'].values
v_wind = data['wnd_vcmp_height_above_ground'].values

# Drop the height dimension
u_wind= np.squeeze(u_wind,axis=1)
v_wind= np.squeeze(v_wind,axis=1)
print(lat.shape, lon.shape, u_wind.shape, v_wind.shape)
# Print variable names
var_name = data['wnd_ucmp_height_above_ground'].name
print(var_name)

In [ ]:
u_wind_on_site = get_values(lon, lat, u_wind[0], var_name, daily_trajectories)
#v_wind_on_site = get_values(lon, lat, v_wind[0], daily_trajectories)
u_wind_on_site

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap

u = u_wind[0,:,:]
v = v_wind[0,:,:]
magnitude = np.sqrt(u**2+v**2)

fig, axes=plt.subplots(figsize=(10,10))
axes.set_title('Wind Speed and Direction in the Pacific Ocean on {}'.format(date_time))
axes.set_xlabel('Longitude')
axes.set_ylabel('Latitude')
axes.set_xlim([-78.5,-77.5])
axes.set_ylim([-12.5,-11.5])


m=Basemap(projection='mill',lat_ts=15,llcrnrlon=lon.min(), \
          urcrnrlon=lon.max(),llcrnrlat=lat.min(),urcrnrlat=lat.max(), \
          resolution='i')

m.fillcontinents(color='0.6')
m.drawcoastlines()
x,y=m(*np.meshgrid(lon,lat))
#===============================
uproj,vproj,xx,yy=m.transform_vector(u,v,lon,lat,20,30,returnxy=True,masked=True)
#===============================

m.pcolormesh(x,y,magnitude,cmap=plt.cm.Reds)
m.quiver(xx,yy,uproj,vproj)
m.drawparallels(np.arange(-18,-2,1),labels=[1,0,0,0],linewidth=0.7,fontsize=9)
m.drawmeridians(np.arange(-86,-70,1),labels=[0,0,0,1],linewidth=0.7,fontsize=9)
m.colorbar()

# PLot the trajectories
x_1,y_1=m(daily_trajectories['longitude'].values,daily_trajectories['latitude'].values)
m.scatter(x_1, y_1, marker='o',color='b',s=10)

plt.show()